# Generating offspring photos with ComfyUI

## 1. Set Up

In [ ]:
!git clone https://github.com/comfyanonymous/ComfyUI.git

In [3]:
!pip install -r requirements.txt

  Using cached einops-0.8.0-py3-none-any.whl.metadata (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.5 MB/s eta 0:00:00
  Using cached sentencepiece-0.2.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.8 MB/s eta 0:00:00
  Using cached tqdm-4.67.0-py3-none-any.whl.metadata (57 kB)
  Using cached soundfile-0.12.1-py2.py3-none-macosx_11_0_arm64.whl.metadata (14 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.9 MB/s eta 0:00:00
  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl.metadata (6.1 kB)
 

In [4]:
import sys
import os
from pathlib import Path

In [5]:
# Define my specific ComfyUI path
COMFY_PATH = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI")

# Verify the path exists
if not COMFY_PATH.exists():
    raise Exception(f"ComfyUI path not found at {COMFY_PATH}")
else:
    print(f"ComfyUI directory found at {COMFY_PATH}")

# Add ComfyUI to system path
if str(COMFY_PATH) not in sys.path:
    sys.path.append(str(COMFY_PATH))
    print("Added ComfyUI to system path")

ComfyUI directory found at /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI
Added ComfyUI to system path


In [6]:
import requests
from tqdm import tqdm

In [7]:
def download_model(url, save_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    save_path = Path(save_path)
    save_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(save_path, 'wb') as f, tqdm(
        desc=save_path.name,
        total=total_size,
        unit='iB',
        unit_scale=True
    ) as pbar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            pbar.update(size)

In [8]:
# Create model directories
models_path = COMFY_PATH / "models"
checkpoints_path = models_path / "checkpoints"
controlnet_path = models_path / "controlnet"

In [9]:
# Create directories and print status
for path in [models_path, checkpoints_path, controlnet_path]:
    path.mkdir(parents=True, exist_ok=True)
    print(f"Created/verified directory: {path}")

Created/verified directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/models
Created/verified directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/models/checkpoints
Created/verified directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/models/controlnet


## 2. Download the Model

The workflow:

Base Model (SD XL Turbo): We're using this because it's:
- Faster than regular Stable Diffusion
- Better at handling facial features
- More consistent with human anatomy

ControlNet Face: This will help:
- Preserve important facial features from both you and the donors
- Maintain consistent facial structure
- Ensure realistic baby face generation


IP-Adapter: This helps:
- Better blend features from multiple images
- Maintain genetic characteristics
- Create more natural-looking results

In [10]:
# Define the models we need with their URLs and purposes
models = {
    "sd_xl_turbo": {
        "url": "https://huggingface.co/stabilityai/sdxl-turbo/resolve/main/sd_xl_turbo_1.0.safetensors",
        "path": checkpoints_path / "sd_xl_turbo_1.0.safetensors",
        "size": "6.9GB",
        "purpose": "Base model - chosen for speed and quality in generating human faces"
    },
    "controlnet_face": {
        "url": "https://huggingface.co/CrucibleAI/ControlNetMediaPipeFace/resolve/main/controlnet-mediapipe-face.safetensors",
        "path": controlnet_path / "controlnet-mediapipe-face.safetensors",
        "size": "1.4GB",
        "purpose": "Ensures accurate facial feature preservation and transformation"
    },
    "ip_adapter": {
        "url": "https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.safetensors",
        "path": models_path / "ip_adapters" / "ip-adapter_sdxl_vit-h.safetensors",
        "size": "0.7GB",
        "purpose": "Helps blend facial features between input images"
    }
}

In [12]:
# Print model information before downloading
print("Models to be downloaded:")
for name, info in models.items():
    print(f"\n{name}:")
    print(f"Size: {info['size']}")
    print(f"Purpose: {info['purpose']}")
    print(f"Will be saved to: {info['path']}")

Models to be downloaded:

sd_xl_turbo:
Size: 6.9GB
Purpose: Base model - chosen for speed and quality in generating human faces
Will be saved to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/models/checkpoints/sd_xl_turbo_1.0.safetensors

controlnet_face:
Size: 1.4GB
Purpose: Ensures accurate facial feature preservation and transformation
Will be saved to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/models/controlnet/controlnet-mediapipe-face.safetensors

ip_adapter:
Size: 0.7GB
Purpose: Helps blend facial features between input images
Will be saved to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/models/ip_adapters/ip-adapter_sdxl_vit-h.safetensors


In [13]:
proceed = input("\nProceed with download? (yes/no): ")

if proceed.lower() == 'yes':
    for model_name, model_info in models.items():
        if not model_info["path"].exists():
            print(f"\nDownloading {model_name}...")
            # Create parent directory if it doesn't exist
            model_info["path"].parent.mkdir(parents=True, exist_ok=True)
            download_model(model_info["url"], model_info["path"])
            print(f"Downloaded {model_name}")
        else:
            print(f"\n{model_name} already exists at {model_info['path']}")
else:
    print("Download cancelled")

sd_xl_turbo_1.0.safetensors: 100%|██████████| 13.9G/13.9G [15:15<00:00, 15.2MiB/s]


Downloaded sd_xl_turbo



controlnet-mediapipe-face.safetensors: 100%|██████████| 15.0/15.0 [00:00<00:00, 109kiB/s]


Downloaded controlnet_face



ip-adapter_sdxl_vit-h.safetensors: 100%|██████████| 698M/698M [00:51<00:00, 13.5MiB/s] 

Downloaded ip_adapter


## Start the ComfyUI server

In [10]:
import subprocess
import webbrowser
from time import sleep

In [11]:
# Start ComfyUI server
server_process = subprocess.Popen(
    ['python', str(COMFY_PATH / 'main.py'), '--listen', '0.0.0.0', '--port', '8188'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait a moment for the server to start
sleep(5)

# Open the UI in default browser
webbrowser.open('http://localhost:8188')

print("ComfyUI server started! The interface should open in your browser.")
print("Keep this notebook running while you use ComfyUI.")

ComfyUI server started! The interface should open in your browser.
Keep this notebook running while you use ComfyUI.


## 3. Setting up Paths

In [12]:
import json
import csv
from datetime import datetime
import shutil
from pathlib import Path

In [13]:
def setup_donor_folders(donor_filename):
    """
    Creates the folder structure for a donor's generations
    """
    donor_stem = Path(donor_filename).stem
    donor_dir = PATHS["output"] / "donor_logs" / donor_stem / "v1"
    donor_dir.mkdir(parents=True, exist_ok=True)
    return donor_dir

def create_generation_metadata(donor_filename, seeds, cfg=7.0, steps=25):
    """
    Creates metadata for a single donor's generation session
    """
    return {
        "donor_image": donor_filename,
        "timestamp": datetime.now().isoformat(),
        "generation_params": {
            "cfg": cfg,
            "steps": steps,
            "sampler": "euler",
        },
        "seeds": seeds,
        "output_files": [
            f"{Path(donor_filename).stem}_baby_{i+1}.png" 
            for i in range(len(seeds))
        ]
    }

def update_main_logs(metadata, csv_path, json_path):
    """
    Updates both CSV and JSON logs with new generation data
    """
    # Update JSON log
    if json_path.exists():
        with open(json_path, 'r') as f:
            all_logs = json.load(f)
    else:
        all_logs = []
    
    all_logs.append(metadata)
    with open(json_path, 'w') as f:
        json.dump(all_logs, f, indent=2)

    # Update CSV log
    csv_exists = csv_path.exists()
    with open(csv_path, 'a', newline='') as f:
        writer = csv.writer(f)
        if not csv_exists:
            writer.writerow([
                'donor_image', 'timestamp', 'cfg', 
                'steps', 'sampler', 'seeds', 'output_files'
            ])
        writer.writerow([
            metadata['donor_image'],
            metadata['timestamp'],
            metadata['generation_params']['cfg'],
            metadata['generation_params']['steps'],
            metadata['generation_params']['sampler'],
            ','.join(map(str, metadata['seeds'])),
            ','.join(metadata['output_files'])
        ])

In [14]:
# Create input directory for my photo
input_path = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input")
input_path.mkdir(parents=True, exist_ok=True)

# Paths dictionary
PATHS = {
    "donors": Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/comfyui_source_files"),
    "output": Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/outputs/generated_offsprings"),
    "my_photo": input_path / "cindy.jpg" 
}

In [15]:
# Verify paths
for name, path in PATHS.items():
    if path.exists():
        print(f"✓ Found {name} directory: {path}")
    else:
        if name == "my_photo":
            print(f"\nPlease save your headshot photo as: {path}")
            print("Once saved, run this cell again to verify.")
        elif name == "output":
            path.mkdir(parents=True, exist_ok=True)
            print(f"Created output directory: {path}")
        else:
            print(f"❌ ERROR: {name} directory not found: {path}")

✓ Found donors directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/comfyui_source_files
✓ Found output directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/outputs/generated_offsprings
✓ Found my_photo directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/cindy.jpg


In [16]:
import shutil
from pathlib import Path

In [17]:
# Paths cleanup (to comply with ComfyUI's output structure)
PATHS = {
    "donors": Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/comfyui_source_files"),
    "output": Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/outputs/generated_offsprings"),
    "my_photo": input_path / "cindy.jpg" 
}

# 1. Clean up: Remove any previously created symlinks or directories
project_outputs = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/thesis_outputs")
if project_outputs.exists():
    if project_outputs.is_symlink():
        project_outputs.unlink()
    elif project_outputs.is_dir():
        shutil.rmtree(project_outputs)

# 2. Set up fresh directory structure in ComfyUI's output
comfy_output = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output")
donor_output = comfy_output / "donor_logs" / "aug_gpt_4_gem_wo" / "v1"
donor_output.mkdir(parents=True, exist_ok=True)

print("Cleaned up previous setup and created fresh directory structure.")
print(f"ComfyUI output directory: {comfy_output}")
print(f"Donor output directory: {donor_output}")

Cleaned up previous setup and created fresh directory structure.
ComfyUI output directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output
Donor output directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/donor_logs/aug_gpt_4_gem_wo/v1


## 4. ComfyUI Workflow
- Testing 1 file

In [18]:
import random

In [19]:
def generate_random_seeds(count=3):
    """Generate list of random seeds for multiple generations"""
    return [random.randint(1, 2**31 - 1) for _ in range(count)]

# Test with our sample donor
test_donor = PATHS["donors"] / "aug_gpt_4_gem_wo.jpeg"
donor_output_dir = setup_donor_folders(test_donor.name)

# Generate seeds for our three variations
seeds = generate_random_seeds(3)

# Create workflow with corrected parameters
updated_workflow = {
    "version": 1.0,
    "state": {},
    "nodes": [
        {
            "id": 1,
            "type": "LoadImage",
            "pos": [50, 200],
            "size": [315, 98],
            "flags": {},
            "order": 0,
            "mode": 0,
            "properties": {"Node name for S&R": "LoadImage"},
            "inputs": [],
            "outputs": [
                {
                    "name": "IMAGE",
                    "type": "IMAGE",
                    "links": [5],
                    "slot_index": 0
                }
            ],
            "widgets_values": [str(PATHS["my_photo"])]
        }
    ]  # Closing the 'nodes' list
}  # Closing the 'updated_workflow' dictionary


In [20]:
# Setup logging paths
csv_log_path = PATHS["output"] / "all_generations_v1.csv"
json_log_path = PATHS["output"] / "all_generations_v1.json"

# Modify workflow for our test donor with the first seed
def create_workflow_config(donor_image, output_dir, seed):
    return {
        "3": {  # CheckpointLoaderSimple node
            "inputs": {},
            "class_type": "CheckpointLoaderSimple",
            "widgets_values": ["sd_xl_turbo_1.0.safetensors"]
        },
        "5": {  # KSampler node
            "inputs": {
                "seed": seed,
                "steps": 25,
                "cfg": 7.0,
                "sampler_name": "euler",
                "scheduler": "normal",
                "denoise": 1.0,
                "model": ["3", 0],
                "positive": ["4", 0],
                "negative": ["9", 0],
                "latent_image": ["8", 0]
            },
            "class_type": "KSampler",
        },
        "7": {  # SaveImage node
            "inputs": {
                "images": ["6", 0],
                "filename_prefix": Path(donor_image).stem + "_baby",
                "filename_separator": "_"
            },
            "class_type": "SaveImage",
            "widgets_values": [str(output_dir), ""]
        }
    }


# Create and save complete workflow for test
test_workflow_path = PATHS["output"] / "workflows" / "test_workflow.json"
test_workflow_path.parent.mkdir(parents=True, exist_ok=True)

# Create metadata for our test generation
test_metadata = create_generation_metadata(
    test_donor.name,
    seeds=seeds,
)

# Save initial metadata
donor_metadata_path = donor_output_dir.parent / "metadata_v1.json"
with open(donor_metadata_path, "w") as f:
    json.dump(test_metadata, f, indent=2)

print(f"Setup complete!")
print(f"Test donor: {test_donor.name}")
print(f"Output directory: {donor_output_dir}")
print(f"Seeds to be used: {seeds}")

Setup complete!
Test donor: aug_gpt_4_gem_wo.jpeg
Output directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/outputs/generated_offsprings/donor_logs/aug_gpt_4_gem_wo/v1
Seeds to be used: [773169767, 1547005295, 1737388557]


## 5. Using `json` workflow to test on 1 donor `aug_gpt_4_gem_wo.jpeg`
- The output workflow json doc will be at: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/outputs/generated_offsprings/donor_logs/aug_gpt_4_gem_wo/workflow.json
- When it generated baby photos successfully, the template is saved as `base_workflow.json` is saved to `/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/user/default/workflows`

In [21]:
import json
import csv
from datetime import datetime
import random
from pathlib import Path
import shutil

In [22]:
# Save the workflow for ComfyUI
workflow_path = PATHS["output"] / "donor_logs" / test_donor.stem / "workflow.json"
workflow_path.parent.mkdir(parents=True, exist_ok=True)

In [23]:
# Utility Functions
def setup_donor_folders(donor_filename):
    """
    Creates the folder structure for a donor's generations
    """
    donor_stem = Path(donor_filename).stem
    donor_dir = PATHS["output"] / "donor_logs" / donor_stem / "v1"
    donor_dir.mkdir(parents=True, exist_ok=True)
    return donor_dir

def create_generation_metadata(donor_filename, seeds, cfg=7.0, steps=25):
    """
    Creates metadata for a single donor's generation session
    """
    return {
        "donor_image": donor_filename,
        "timestamp": datetime.now().isoformat(),
        "generation_params": {
            "cfg": cfg,
            "steps": steps,
            "sampler": "euler",
        },
        "seeds": seeds,
        "output_files": [
            f"{Path(donor_filename).stem}_baby_{i+1}.png" 
            for i in range(len(seeds))
        ]
    }

def update_main_logs(metadata, csv_path, json_path):
    """
    Updates both CSV and JSON logs with new generation data
    """
    # Update JSON log
    if json_path.exists():
        with open(json_path, 'r') as f:
            all_logs = json.load(f)
    else:
        all_logs = []
    
    all_logs.append(metadata)
    with open(json_path, 'w') as f:
        json.dump(all_logs, f, indent=2)

    # Update CSV log
    csv_exists = csv_path.exists()
    with open(csv_path, 'a', newline='') as f:
        writer = csv.writer(f)
        if not csv_exists:
            writer.writerow([
                'donor_image', 'timestamp', 'cfg', 
                'steps', 'sampler', 'seeds', 'output_files'
            ])
        writer.writerow([
            metadata['donor_image'],
            metadata['timestamp'],
            metadata['generation_params']['cfg'],
            metadata['generation_params']['steps'],
            metadata['generation_params']['sampler'],
            ','.join(map(str, metadata['seeds'])),
            ','.join(metadata['output_files'])
        ])


In [34]:
# Setup for test generation
# Generate random seeds
seeds = [random.randint(1, 2**31 - 1) for _ in range(3)]

# Test with our sample donor
test_donor = PATHS["donors"] / "aug_gpt_4_gem_wo.jpeg"
donor_output_dir = setup_donor_folders(test_donor.name)

# Setup logging paths
csv_log_path = PATHS["output"] / "all_generations_v1.csv"
json_log_path = PATHS["output"] / "all_generations_v1.json"

# Create the complete workflow JSON
complete_workflow = {
    "version": 1.0,
    "state": {},
    "nodes": [
        {
            "id": 1,
            "type": "LoadImage",
            "pos": [50, 200],
            "size": [315, 98],
            "flags": {},
            "order": 0,
            "mode": 0,
            "properties": {"Node name for S&R": "LoadImage"},
            "inputs": [],
            "outputs": [
                {
                    "name": "IMAGE",
                    "type": "IMAGE",
                    "links": [5],
                    "slot_index": 0
                },
                {
                    "name": "MASK",
                    "type": "MASK",
                    "links": [],
                    "slot_index": 1
                }
            ],
            "widgets_values": [str(PATHS["my_photo"])]
        },
        {
            "id": 2,
            "type": "LoadImage",
            "pos": [50, 400],
            "size": [315, 98],
            "flags": {},
            "order": 1,
            "mode": 0,
            "properties": {"Node name for S&R": "LoadImage"},
            "inputs": [],
            "outputs": [
                {
                    "name": "IMAGE",
                    "type": "IMAGE",
                    "links": [6],
                    "slot_index": 0
                },
                {
                    "name": "MASK",
                    "type": "MASK",
                    "links": [],
                    "slot_index": 1
                }
            ],
            "widgets_values": [str(test_donor)]
        },
        {
            "id": 3,
            "type": "CheckpointLoaderSimple",
            "pos": [50, 0],
            "size": [300, 100],
            "flags": {},
            "order": 2,
            "mode": 0,
            "properties": {"Node name for S&R": "CheckpointLoaderSimple"},
            "inputs": [],
            "outputs": [
                {"name": "MODEL", "type": "MODEL", "links": [7], "slot_index": 0},
                {"name": "CLIP", "type": "CLIP", "links": [8, 15], "slot_index": 1},
                {"name": "VAE", "type": "VAE", "links": [9], "slot_index": 2}
            ],
            "widgets_values": ["sd_xl_turbo_1.0.safetensors"]
        },
        {
            "id": 4,
            "type": "CLIPTextEncode",
            "pos": [400, 0],
            "size": [400, 100],
            "flags": {},
            "order": 3,
            "mode": 0,
            "properties": {"Node name for S&R": "CLIPTextEncode"},
            "inputs": [
                {"name": "clip", "type": "CLIP", "link": 8, "slot_index": 0}
            ],
            "outputs": [
                {"name": "CONDITIONING", "type": "CONDITIONING", "links": [10], "slot_index": 0}
            ],
            "widgets_values": ["professional passport photo of a healthy 12 month old baby, front facing portrait, clean natural lighting, clear face, soft natural skin, white background, sharp focus, high quality photograph, Canon camera, photorealistic, no editing, neutral expression"]
        },
        {
            "id": 5,
            "type": "KSampler",
            "pos": [800, 200],
            "size": [400, 400],
            "flags": {},
            "order": 4,
            "mode": 0,
            "properties": {"Node name for S&R": "KSampler"},
            "inputs": [
                {"name": "model", "type": "MODEL", "link": 7, "slot_index": 0},
                {"name": "positive", "type": "CONDITIONING", "link": 10, "slot_index": 1},
                {"name": "negative", "type": "CONDITIONING", "link": 11, "slot_index": 2},
                {"name": "latent_image", "type": "LATENT", "link": 12, "slot_index": 3}
            ],
            "outputs": [
                {"name": "LATENT", "type": "LATENT", "links": [13], "slot_index": 0}
            ],
            "widgets_values": [
                seeds[0],                # seed
                "randomize",            # control_after_generate
                30,                     # steps
                7.5,                    # cfg
                "euler_ancestral",                # sampler_name
                "normal",               # scheduler
                0.8                     # denoise
            ]
        },
        {
            "id": 8,
            "type": "EmptyLatentImage",
            "pos": [600, 300],
            "size": [315, 98],
            "flags": {},
            "order": 3,
            "mode": 0,
            "properties": {"Node name for S&R": "EmptyLatentImage"},
            "inputs": [],
            "outputs": [
                {"name": "LATENT", "type": "LATENT", "links": [12], "slot_index": 0}
            ],
            "widgets_values": [512, 512, 1]
        },
        {
            "id": 9,
            "type": "CLIPTextEncode",
            "pos": [400, 150],
            "size": [400, 100],
            "flags": {},
            "order": 3,
            "mode": 0,
            "properties": {"Node name for S&R": "CLIPTextEncode"},
            "inputs": [
                {"name": "clip", "type": "CLIP", "link": 15, "slot_index": 0}
            ],
            "outputs": [
                {"name": "CONDITIONING", "type": "CONDITIONING", "links": [11], "slot_index": 0}
            ],
            "widgets_values": ["distorted, deformed, ugly, mutation, blurry, grainy, noisy, oversaturated, overprocessed, artificial, harsh lighting, unrealistic colors, plastic skin, extreme contrast, overexposed, surreal, watercolor, painting, illustration, artificial HDR, oversharpened"]
        },
        {
            "id": 6,
            "type": "VAEDecode",
            "pos": [1200, 200],
            "size": [200, 100],
            "flags": {},
            "order": 5,
            "mode": 0,
            "properties": {"Node name for S&R": "VAEDecode"},
            "inputs": [
                {"name": "samples", "type": "LATENT", "link": 13, "slot_index": 0},
                {"name": "vae", "type": "VAE", "link": 9, "slot_index": 1}
            ],
            "outputs": [
                {"name": "IMAGE", "type": "IMAGE", "links": [14], "slot_index": 0}
            ]
        },
        {
            "id": 7,
            "type": "SaveImage",
            "pos": [1400, 200],
            "size": [400, 100],
            "flags": {},
            "order": 8,
            "mode": 0,
            "inputs": [{"name": "images", "type": "IMAGE", "link": 14, "slot_index": 0}],
            "outputs": [],
            "properties": {"Node name for S&R": "SaveImage"},
            "widgets_values": ["aug_gpt_4_gem_wo_baby_1"]  # Just the filename, no path
        }
    ],
    "links": [
        {"id": 5, "origin_id": 1, "origin_slot": 0, "target_id": 5, "target_slot": 0, "type": "IMAGE"},
        {"id": 6, "origin_id": 2, "origin_slot": 0, "target_id": 5, "target_slot": 1, "type": "IMAGE"},
        {"id": 7, "origin_id": 3, "origin_slot": 0, "target_id": 5, "target_slot": 0, "type": "MODEL"},
        {"id": 8, "origin_id": 3, "origin_slot": 1, "target_id": 4, "target_slot": 0, "type": "CLIP"},
        {"id": 9, "origin_id": 3, "origin_slot": 2, "target_id": 6, "target_slot": 1, "type": "VAE"},
        {"id": 10, "origin_id": 4, "origin_slot": 0, "target_id": 5, "target_slot": 1, "type": "CONDITIONING"},
        {"id": 11, "origin_id": 9, "origin_slot": 0, "target_id": 5, "target_slot": 2, "type": "CONDITIONING"},
        {"id": 12, "origin_id": 8, "origin_slot": 0, "target_id": 5, "target_slot": 3, "type": "LATENT"},
        {"id": 13, "origin_id": 5, "origin_slot": 0, "target_id": 6, "target_slot": 0, "type": "LATENT"},
        {"id": 14, "origin_id": 6, "origin_slot": 0, "target_id": 7, "target_slot": 0, "type": "IMAGE"},
        {"id": 15, "origin_id": 3, "origin_slot": 1, "target_id": 9, "target_slot": 0, "type": "CLIP"}
    ]
}



In [35]:
# Save the workflow
workflow_path = donor_output_dir.parent / "workflow.json"
with open(workflow_path, "w") as f:
    json.dump(complete_workflow, f, indent=2)

# Create metadata for our test generation
test_metadata = create_generation_metadata(
    test_donor.name,
    seeds=seeds,
)

# Save initial metadata
donor_metadata_path = donor_output_dir.parent / "metadata_v1.json"
with open(donor_metadata_path, "w") as f:
    json.dump(test_metadata, f, indent=2)

# Update main logs
update_main_logs(test_metadata, csv_log_path, json_log_path)

print(f"\nSetup complete!")
print(f"Test donor: {test_donor.name}")
print(f"Output directory: {donor_output_dir}")
print(f"Seeds to be used: {seeds}")
print(f"\nWorkflow saved to: {workflow_path}")
print("\nTo generate images:")
print("1. Go to ComfyUI interface at http://localhost:8188")
print("2. Click 'Load' and select the workflow file at:", workflow_path)
print("3. Click 'Queue' to generate the first baby photo")
print("\nAfter first generation completes:")
print("4. Change the seed to", seeds[1], "and filename to '_baby_2'")
print("5. Generate again")
print("6. Change the seed to", seeds[2], "and filename to '_baby_3'")
print("7. Generate one final time")


Setup complete!
Test donor: aug_gpt_4_gem_wo.jpeg
Output directory: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/outputs/generated_offsprings/donor_logs/aug_gpt_4_gem_wo/v1
Seeds to be used: [2103594951, 1025833320, 95593290]

Workflow saved to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/outputs/generated_offsprings/donor_logs/aug_gpt_4_gem_wo/workflow.json

To generate images:
1. Go to ComfyUI interface at http://localhost:8188
2. Click 'Load' and select the workflow file at: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/outputs/generated_offsprings/donor_logs/aug_gpt_4_gem_wo/workflow.json
3. Click 'Queue' to generate the first baby photo

After first generation completes:
4. Change the seed to 1025833320 and filename to '_baby_2'
5. Generate again
6. Change the seed to 95593290 and filename to '_baby_3'
7. Generate one final time


## 6. Batch Processing
Testing with 5 donors first to ensure everything works properly

In [36]:
# Import and Setup
import json
import time
from pathlib import Path
import shutil
import random

In [40]:
# Update paths
COMFY_PATH = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI")
COMFY_OUTPUT = COMFY_PATH / "output"
DONORS_PATH = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/comfyui_source_files")
MY_PHOTO = Path("/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/input/cindy.jpg")
BASE_WORKFLOW_PATH = COMFY_PATH / "user/default/workflows/base_workflow.json"

In [41]:
# Utility Functions
def verify_saved_image(output_path: Path, donor_id: str, variation: int) -> bool:
    """Verify that image was saved and is valid"""
    expected_file = list(output_path.glob(f"{donor_id}_baby_{variation}*.png"))
    
    if not expected_file:
        print(f"❌ Warning: No output file found for {donor_id} variation {variation}")
        return False
        
    if expected_file[0].stat().st_size == 0:
        print(f"❌ Warning: Empty file generated for {donor_id} variation {variation}")
        return False
        
    print(f"✓ Successfully saved: {expected_file[0].name}")
    return True

def generate_random_seeds(count=3):
    """Generate list of random seeds for multiple generations"""
    return [random.randint(1, 2**31 - 1) for _ in range(count)]


In [42]:
# Load and modify base workflow
print(f"Loading base workflow from: {BASE_WORKFLOW_PATH}")
if not BASE_WORKFLOW_PATH.exists():
    raise FileNotFoundError(f"Base workflow not found at: {BASE_WORKFLOW_PATH}")

with open(BASE_WORKFLOW_PATH, 'r') as f:
    base_workflow = json.load(f)
print("Successfully loaded base workflow")

def modify_workflow_for_donor(workflow: dict, donor_path: str, seed: int, variation: int) -> dict:
    """Modify workflow for specific donor and variation"""
    modified = workflow.copy()
    
    # Update LoadImage nodes
    for node in modified['nodes']:
        if node['type'] == 'LoadImage':
            if node['id'] == 1:  # Your photo
                node['widgets_values'] = [str(MY_PHOTO)]
            elif node['id'] == 2:  # Donor photo
                node['widgets_values'] = [str(donor_path)]
        
        # Update KSampler parameters
        elif node['type'] == 'KSampler':
            node['widgets_values'] = [
                seed,           # seed
                "randomize",    # control_after_generate
                30,            # steps
                7.5,           # cfg
                "euler",       # sampler_name
                "normal",      # scheduler
                0.8            # denoise
            ]
        
        # Update SaveImage filename
        elif node['type'] == 'SaveImage':
            donor_id = Path(donor_path).stem
            node['widgets_values'] = [f"{donor_id}_baby_{variation}"]
    
    return modified


Loading base workflow from: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/user/default/workflows/base_workflow.json
Successfully loaded base workflow


### Test Batch Processing Setup

In [43]:
# Get first 5 donors
test_donors = sorted(list(DONORS_PATH.glob("*.jpeg")))[:5]

print("\nWill process these donors:")
for i, donor in enumerate(test_donors, 1):
    print(f"{i}. {donor.name}")



Will process these donors:
1. aug_cla_2_gem_wo.jpeg
2. aug_cla_3_gem_wo.jpeg
3. aug_cla_4_gem_wo.jpeg
4. aug_cla_5_gem_wo.jpeg
5. aug_cla_6_gem_demo.jpeg


### Process Test Batch

In [45]:
 def process_test_batch():
    total_donors = len(test_donors)
    
    for donor_idx, donor_path in enumerate(test_donors, 1):
        print(f"\n{'='*50}")
        print(f"Processing donor {donor_idx}/{total_donors}: {donor_path.name}")
        
        # Generate 3 variations for this donor
        seeds = generate_random_seeds(3)
        
        for variation in range(1, 4):
            print(f"\nGenerating variation {variation}/3")
            
            # Modify workflow for this variation
            workflow = modify_workflow_for_donor(
                base_workflow,
                str(donor_path),
                seeds[variation-1],
                variation
            )
            
            # Save workflow for this variation
            workflow_path = COMFY_OUTPUT / "workflows" / f"{donor_path.stem}_v{variation}.json"
            workflow_path.parent.mkdir(parents=True, exist_ok=True)
            
            with open(workflow_path, 'w') as f:
                json.dump(workflow, f, indent=2)
            
            print(f"✓ Saved workflow to: {workflow_path}")
            print(f"⚠️ Please load this workflow in ComfyUI and click 'Queue'")
            
            # Wait for user confirmation before continuing
            input("Press Enter after the image has been generated...")
            
            # Verify saved image
            verify_saved_image(COMFY_OUTPUT, donor_path.stem, variation)
            
            # Brief pause between generations
            time.sleep(2)
        
        print(f"\n✓ Completed all variations for {donor_path.name}")
    
    print("\n🎉 Batch processing complete!")

# Run with:
process_test_batch()


Processing donor 1/5: aug_cla_2_gem_wo.jpeg

Generating variation 1/3
✓ Saved workflow to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/workflows/aug_cla_2_gem_wo_v1.json
⚠️ Please load this workflow in ComfyUI and click 'Queue'
✓ Successfully saved: aug_cla_2_gem_wo_baby_1_00001_.png

Generating variation 2/3
✓ Saved workflow to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/workflows/aug_cla_2_gem_wo_v2.json
⚠️ Please load this workflow in ComfyUI and click 'Queue'
✓ Successfully saved: aug_cla_2_gem_wo_baby_2_00001_.png

Generating variation 3/3
✓ Saved workflow to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/models/ComfyUI/output/workflows/aug_cla_2_gem_wo_v3.json
⚠️ Please load this workflow in ComfyUI and click 'Queue'
✓ Successfully saved: aug_cla_2_gem_wo_baby_3_00001_.png

✓ Completed all variations for aug_cla_2_gem_wo.jpeg

Processing donor 2/5: aug_cla_3_gem_wo.jpeg

Genera

## 7. Automated Batch Processing
Setting up automated generation using ComfyUI's WebSocket API

In [66]:
# Import additional libraries
import requests
import json
import time
from pathlib import Path
import random
from datetime import datetime
import asyncio

In [67]:
class ComfyUIAutomator:
    def __init__(self, host="127.0.0.1", port=8188):
        self.host = host
        self.port = port
        self.base_url = f"http://{host}:{port}"
        
    def verify_server(self):
        """Verify ComfyUI server is running and accessible"""
        try:
            response = requests.get(f"{self.base_url}/history")
            if response.status_code == 200:
                print("✓ ComfyUI server is running and accessible")
                return True
            else:
                print("❌ ComfyUI server returned unexpected status code:", response.status_code)
                return False
        except requests.exceptions.ConnectionError:
            print("❌ Could not connect to ComfyUI server. Is it running?")
            return False

    def queue_prompt(self, workflow):
        """Queue a prompt using HTTP API"""
        try:
            # Queue the prompt
            response = requests.post(f"{self.base_url}/prompt", json=workflow)
            if response.status_code != 200:
                print(f"❌ Failed to queue prompt: {response.status_code}")
                return False
                
            prompt_id = response.json()['prompt_id']
            print(f"✓ Prompt queued (ID: {prompt_id})")
            
            # Wait for generation to complete
            while True:
                status_response = requests.get(f"{self.base_url}/history")
                if status_response.status_code != 200:
                    print("❌ Failed to get status")
                    return False
                    
                history = status_response.json()
                if prompt_id in history:
                    if 'outputs' in history[prompt_id]:
                        print("✓ Generation completed")
                        return True
                    elif 'error' in history[prompt_id]:
                        print(f"❌ Generation failed: {history[prompt_id]['error']}")
                        return False
                        
                print("⏳ Waiting for generation...")
                time.sleep(2)
                
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            return False

def process_batch(donors, base_workflow):
    automator = ComfyUIAutomator()
    batch_start = time.time()
    
    for donor_idx, donor_path in enumerate(donors, 1):
        donor_start = time.time()
        print(f"\n{'='*50}")
        print(f"[{datetime.now().strftime('%H:%M:%S')}] Processing donor {donor_idx}/{len(donors)}: {donor_path.name}")
        
        for variation in range(1, 4):
            print(f"\nGenerating variation {variation}/3")
            
            # Modify workflow
            workflow = base_workflow.copy()
            for node in workflow['nodes']:
                if node['type'] == 'KSampler':
                    node['widgets_values'] = [
                        random.randint(1, 2**31 - 1),  # random seed
                        "randomize",
                        30,                    # steps
                        7.5,                   # cfg
                        "euler",               # sampler
                        "normal",
                        0.8                    # denoise
                    ]
                elif node['type'] == 'SaveImage':
                    node['widgets_values'] = [
                        f"{Path(donor_path).stem}_baby_{variation}",
                        ""  # prevent auto-numbering
                    ]
            
            # Queue and wait for completion
            success = automator.queue_prompt(workflow)
            if not success:
                print(f"❌ Failed to generate variation {variation}")
            else:
                print(f"✓ Successfully generated variation {variation}")
            
            # Brief pause between generations
            time.sleep(1)
        
        donor_elapsed = time.time() - donor_start
        print(f"✓ Completed donor {donor_path.name} in {donor_elapsed:.1f}s")
    
    batch_elapsed = time.time() - batch_start
    print(f"\n🎉 Batch processing completed in {batch_elapsed:.1f}s")


In [68]:
# Test Function
def run_test(num_donors=1):
    # Create automator instance
    automator = ComfyUIAutomator()
    
    # Verify server is running
    if not automator.verify_server():
        print("Please make sure ComfyUI server is running first!")
        return
    
    # Load base workflow
    try:
        with open(BASE_WORKFLOW_PATH, 'r') as f:
            base_workflow = json.load(f)
    except Exception as e:
        print(f"❌ Error loading workflow: {str(e)}")
        return
    
    # Get specified number of donors
    test_donors = sorted(list(DONORS_PATH.glob("*.jpeg")))[:num_donors]
    
    print(f"Testing automation with {num_donors} donor(s):")
    for donor in test_donors:
        print(f"- {donor.name}")
    
    # Process batch
    process_batch(test_donors, base_workflow)



In [69]:
# Run test
print(f"[{datetime.now().strftime('%H:%M:%S')}] Starting test generation...")
run_test(num_donors=1)

[20:48:07] Starting test generation...
✓ ComfyUI server is running and accessible
Testing automation with 1 donor(s):
- aug_cla_2_gem_wo.jpeg

[20:48:07] Processing donor 1/1: aug_cla_2_gem_wo.jpeg

Generating variation 1/3
❌ Failed to queue prompt: 400
❌ Failed to generate variation 1

Generating variation 2/3
❌ Failed to queue prompt: 400
❌ Failed to generate variation 2

Generating variation 3/3
❌ Failed to queue prompt: 400
❌ Failed to generate variation 3
✓ Completed donor aug_cla_2_gem_wo.jpeg in 3.0s

🎉 Batch processing completed in 3.0s
